In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import pylab as plt
import seaborn as sns
import re
import warnings
warnings.filterwarnings('ignore') 
#para que salga el grafico
#pd.set_option('display.max_rows', None)
%matplotlib inline
from statistics import mean
import pylab as plt
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as RFR  
from sklearn.tree import ExtraTreeRegressor as ETR
from sklearn.ensemble import GradientBoostingRegressor as GBR
from xgboost import XGBRegressor as XGBR
from catboost import CatBoostRegressor as CTR
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.linear_model import Lasso    
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet 
from sklearn.model_selection import train_test_split as tts   
from sklearn.metrics import mean_squared_error as mse  

In [2]:
unpopular = pd.read_csv('../Spoti/unpopular_songs.csv')

unpopular.head(5)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,explicit,popularity,track_name,track_artist,track_id
0,0.530,0.770,4,-6.633,0,0.0389,0.284,0.501000,0.744,0.623,120.144,225696,False,2,No Regrets,James Reeder,6f2c4a9lNx8aowZJngv7cJ
1,0.565,0.730,1,-6.063,1,0.0730,0.365,0.000000,0.237,0.511,130.026,158093,False,2,Wild Life,James Reeder,3fTs52jsDzSuVLsifxNKO8
2,0.427,0.546,4,-8.727,1,0.0849,0.539,0.015200,0.368,0.435,78.345,167262,False,2,Fangs,James Reeder,6NPafqavrv0icaIHMQnXDy
3,0.421,0.531,7,-5.516,1,0.0262,0.706,0.000208,0.110,0.383,85.080,236832,False,2,Afterburner,James Reeder,3vGmhxveURgmlZStvo0uc1
4,0.537,0.804,8,-7.378,0,0.1570,0.379,0.000489,0.323,0.543,139.950,239400,False,2,Hellfire Rising,James Reeder,4O2qRbfCHzMMgfbw9DBdGf


In [3]:
unpopular.drop(['explicit','mode','popularity','key','track_id','instrumentalness'],axis=1,inplace=True)

In [4]:
unpopular=unpopular.iloc[:, [10,9,1,0,2,4,3,5,6,7,8]]


In [5]:
unpopular.rename(columns = {'track_artist':'artist_name', 'track_name':'track_name','duration_ms':'duration'}, inplace = True)

In [6]:
unpopular['duration'] = (unpopular['duration']/1000).round(2)

unpopular.head()

,artist_name,track_name,energy,danceability,loudness,acousticness,speechiness,liveness,valence,tempo,duration
0,James Reeder,No Regrets,0.770,0.530,-6.633,0.284,0.0389,0.744,0.623,120.144,225.70
1,James Reeder,Wild Life,0.730,0.565,-6.063,0.365,0.0730,0.237,0.511,130.026,158.09
2,James Reeder,Fangs,0.546,0.427,-8.727,0.539,0.0849,0.368,0.435,78.345,167.26
3,James Reeder,Afterburner,0.531,0.421,-5.516,0.706,0.0262,0.110,0.383,85.080,236.83
4,James Reeder,Hellfire Rising,0.804,0.537,-7.378,0.379,0.1570,0.323,0.543,139.950,239.40


In [7]:
unpopular.drop_duplicates().shape==unpopular.shape    

False

In [8]:
unpopular.shape

(10877, 11)

In [9]:
unpopular=unpopular.drop_duplicates()

In [10]:
unpopular.shape

(10859, 11)

# Encima de esto he quitado duplicados para una cosa que queria comprobar aunque no era el proposito de este jupyter.

In [ ]:
unpopular.describe().T

In [ ]:
stats=unpopular.describe().T
stats['IQR']=stats['75%']-stats['25%']
outliers=pd.DataFrame(columns=stats.index)
for c in stats.index:
    iqr=stats.at[c, 'IQR']
    cutoff=1.5 * iqr   # test de tukey
    lower=stats.at[c, '25%'] - cutoff
    upper=stats.at[c, '75%'] + cutoff
    res=unpopular[(unpopular[c] < lower) | (unpopular[c] > upper)].copy()
    res['outliers']=c
    outliers=outliers.append(res, sort=True)
outliers['outliers'].value_counts()
clean_index=[i for i in unpopular.index if i not in outliers.index]
unpopular=unpopular.iloc[clean_index]

In [ ]:
unpopular.shape

In [ ]:
unpopular.duration.describe().T

In [ ]:
unpopular.to_csv('../Proyecto-FInal-Spotify/CSV_primeros/unpopular_outliers.csv', index=False)

In [ ]:
unpopular.head()

In [ ]:
unpopular['top_hit']=[0 for i in range(len(unpopular))]


In [ ]:
unpopular.shape